# Tarea 2: Procesamiento de Lenguaje Natural



Integrantes:

* Sebastián Alday (Rut: 18294095-K)
* Paula Canales G. (Rut: 18845155-1)
* Álvaro Neira R. (Rut: 13757209-5)
* Matias Rodriguez U. (Rut: 18362815-1)

# Introducción

# 1. Creacion y limpieza de la base de datos 

## 1.1 Elija una organización que cuenta con una lista de preguntas y respuestas

Chilexpress
https://centrodeayuda.chilexpress.cl/home

## 1.2 Genere una tabla (tablaQA.xls) con las preguntas y respuestas de la organización seleccionada.

In [ ]:
!wget https://www.dropbox.com/s/9zty4fuh8boofda/tablaQA.csv

--2021-12-29 12:05:29--  https://www.dropbox.com/s/9zty4fuh8boofda/tablaQA.csv
Resolving www.dropbox.com (www.dropbox.com)... 162.125.3.18, 2620:100:601b:18::a27d:812
Connecting to www.dropbox.com (www.dropbox.com)|162.125.3.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/9zty4fuh8boofda/tablaQA.csv [following]
--2021-12-29 12:05:29--  https://www.dropbox.com/s/raw/9zty4fuh8boofda/tablaQA.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc4b6d713e46d7758d36e4ce97f9.dl.dropboxusercontent.com/cd/0/inline/Bcyo95Ct2BjUVDpvW6xUzwBBFR_AomExjNHahITzOGHMgCPc9y_yV4wQu54y9273WCmo3ohtyqBRjuryeEFx086NoAkn0tyEKDuyVCEujGWd-q4rnSH3tEdpm1DjecXq8p11JRId0G6YLZrdM9DF2LuJ/file# [following]
--2021-12-29 12:05:30--  https://uc4b6d713e46d7758d36e4ce97f9.dl.dropboxusercontent.com/cd/0/inline/Bcyo95Ct2BjUVDpvW6xUzwBBFR_AomExjNHahITzOGHMgCPc9y_yV4wQu54y9273WCmo3ohtyqBRjuryeEFx086NoAkn0

In [ ]:
import pandas as pd
qa = pd.read_csv('/content/tablaQA.csv', sep=';')
qa.head()

,Pregunta,Respuesta
0,¿Qué es la Orden de Transporte (OT)?,La Orden de Transporte (OT) es el número asoci...
1,¿Cuál es el plazo para retirar un envío desde ...,El plazo para retirar un envío desde una sucur...
2,¿Puede otra persona retirar mi envío de una su...,Para que una persona que no es el destinatario...
3,¿Se puede modificar la dirección de envío?,"Sí, el cambio puede ser solicitado por el remi..."
4,¿Puedo cambiar el destino del envío desde una ...,"Sí, el cambio puede ser solicitado por el remi..."


In [ ]:
qa.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46 entries, 0 to 45
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Pregunta   46 non-null     object
 1   Respuesta  46 non-null     object
dtypes: object(2)
memory usage: 864.0+ bytes


## 1.3 Genere una tabla (tiposmensajes.xls) con ejemplos de las siguientes clases de mensaje: "saludo", "despedida", "nombre", "informacion"

In [ ]:
!wget https://www.dropbox.com/s/wqmpjpdin6jmp3a/tiposmensajes.csv

--2021-12-29 12:05:30--  https://www.dropbox.com/s/wqmpjpdin6jmp3a/tiposmensajes.csv
Resolving www.dropbox.com (www.dropbox.com)... 162.125.3.18, 2620:100:601b:18::a27d:812
Connecting to www.dropbox.com (www.dropbox.com)|162.125.3.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/wqmpjpdin6jmp3a/tiposmensajes.csv [following]
--2021-12-29 12:05:30--  https://www.dropbox.com/s/raw/wqmpjpdin6jmp3a/tiposmensajes.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc3d0bb6f68d8453dc2f50bb7767.dl.dropboxusercontent.com/cd/0/inline/Bcx-78bkmZ552i4yaZFBDFIU7rSOUd1woCaU8farFLews4gqr03mixpLK-IXli-_AnCKy7PZHT6Y7UZCBqM2lF77gYlq4rRq-sDtLbcp3SHsFcDJ6IEYBw1w4HKMKwPEsSP2CZEPBlqccyj1dwJIPm5e/file# [following]
--2021-12-29 12:05:31--  https://uc3d0bb6f68d8453dc2f50bb7767.dl.dropboxusercontent.com/cd/0/inline/Bcx-78bkmZ552i4yaZFBDFIU7rSOUd1woCaU8farFLews4gqr03mixpLK-IXli-_AnCKy7PZHT6

In [ ]:
messages = pd.read_csv('/content/tiposmensajes.csv', sep=',')
messages.head()

,Clase,Mensaje
0,saludo,Hola
1,saludo,Hi
2,saludo,Buenos días
3,saludo,Buenas tardes
4,saludo,Como estas


## 1.4 Genere respuestas predeterminadas (respuestasDefecto.xls) para los mensajes de tipo "saludo", "despedida", "nombre"

In [ ]:
!wget https://www.dropbox.com/s/kukxrqpjxi0l4ei/respuestasDefecto.csv

--2021-12-29 12:05:31--  https://www.dropbox.com/s/kukxrqpjxi0l4ei/respuestasDefecto.csv
Resolving www.dropbox.com (www.dropbox.com)... 162.125.3.18, 2620:100:601b:18::a27d:812
Connecting to www.dropbox.com (www.dropbox.com)|162.125.3.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/kukxrqpjxi0l4ei/respuestasDefecto.csv [following]
--2021-12-29 12:05:31--  https://www.dropbox.com/s/raw/kukxrqpjxi0l4ei/respuestasDefecto.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc6928b2ed6bc665061c4112b9bd.dl.dropboxusercontent.com/cd/0/inline/BcwFMVVj94ZhAYFKmpuUrLZr3l14h72EGabgpNuu5ADif8_IEzbAU6XPLU-T-AVIkvN6qHr8N-UW84sRnwVbbdv-qUNVzTq01Dxd-WNHPTPrK2RcL-qoXYA5kuP-UvIVWq0UhUM8qz44nBtDjkRTXKdh/file# [following]
--2021-12-29 12:05:31--  https://uc6928b2ed6bc665061c4112b9bd.dl.dropboxusercontent.com/cd/0/inline/BcwFMVVj94ZhAYFKmpuUrLZr3l14h72EGabgpNuu5ADif8_IEzbAU6XPLU-T-AV

In [ ]:
answers = pd.read_csv('/content/respuestasDefecto.csv', sep=',')
answers.head()

,Clase,Mensaje
0,saludo,"Hola, en qué puedo ayudarte?"
1,saludo,"Hola, cuentame qué necetias"
2,saludo,"Buenos días, en qué puedo ayudarte?"
3,saludo,"Buenas tardes, en qué puedo ayudarte?"
4,saludo,"Un gusto saludar, en qué puedo ayudarte?"


## 1.5 Describa en términos generales las tablas que construyó.



*   Tabla QA: Se construyó en base a las preguntas y respuestas del centro de ayuda de Chilexpress https://centrodeayuda.chilexpress.cl/home . Se extrajo cada pregunta y respuesta de cada categoría (Envíos Nacionales, Envíos y retiros en App, Envíos internacionales, Giros en Chile y el mundo, Atención en sucursal, Cuenta empresa) y se eliminaron las preguntas repetidas. 
*   Tipos de mensajes: se construyó con mensajes tipo de saludo, despedida y nombre. Para la información se utilizó las preguntas obtenidas en la Tabla QA.
*   Respuestas por defecto: se construyó con respuestas tipo de saludo, despedida y nombre de nuestro bot Eva



# 2. Análisis de distancia

Para el primer chatbot utilizará una métrica de su elección para responder
a cada pregunta/texto del usuario.

## 2.1 Utilice algún embeddings utilizado en el curso para codificar el texto de entrada

### 2.1.1 Word2Vec

In [ ]:
import nltk
import gensim
import re
import pandas as pd
import sklearn
import numpy as np

import sklearn.linear_model
import sklearn.model_selection
from sklearn.svm import LinearSVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

In [ ]:
def normalizer(text): #normalizes a given string to lowercase and changes all vowels to their base form
    text = text.lower() #string lowering
    text = re.sub(r'[^A-Za-zñáéíóú]', ' ', text) #replaces every punctuation with a space
    text = re.sub('á', 'a', text) #replaces special vowels to their base forms
    text = re.sub('é', 'e', text)
    text = re.sub('í', 'i', text)
    text = re.sub('ó', 'o', text)
    text = re.sub('ú', 'u', text)
    return text

def preprocessor(text):
  text = normalizer(text)
  tokens = nltk.tokenize.casual_tokenize(text)
  #if len(tokens)==1:
  #  tokens=text
  return tokens

def vectorizer(text, model): #returns a vector representation from a list of words and a given model
    vectors = []
    for i in text:
        try:
            vectors.append(model.wv[i])
        except:
            pass
    return(np.nan_to_num(np.mean(vectors,axis=0)))

In [ ]:
corpus = messages.Mensaje.tolist()
corpus_preprocessed = list(map(preprocessor,corpus))


In [ ]:
model = gensim.models.word2vec.Word2Vec(sentences = corpus_preprocessed, min_count=1)

## 2.2 Con el texto de entrada codificado y usando embeddings proponga una manera de identificar la clase del texto de entrada.

In [ ]:
def predict_class(question):
  tokens = preprocessor(question)
  vector = vectorizer(tokens,model).reshape(1, -1)
  return clf.predict(vector)

### 2.2.1 Word2Vec

In [ ]:
features = np.zeros(shape=(len(messages),model.wv.vectors.shape[1]))
for i,msg in enumerate(messages.Mensaje):
  features[i,:] = vectorizer(preprocessor(msg),model)

cv_results = sklearn.model_selection.cross_validate(sklearn.linear_model.LogisticRegression(max_iter=10000),features,messages.Clase)
cv_results["test_score"].mean()

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  UserWarning,


0.7666666666666667

In [ ]:
clf = make_pipeline(StandardScaler(),LinearSVC(random_state=0, tol=1e-5))
clf.fit(features,np.array(messages.Clase))

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('linearsvc', LinearSVC(random_state=0, tol=1e-05))])

In [ ]:
print(predict_class('¿Sólo las empresas pueden tener una cuenta con Chilexpress?'))
print(predict_class('Mi nombre es'))
print(predict_class('hola buenas'))
print(predict_class('hola'))
print(predict_class('¿puedo retirar un giro de dinero?'))
print(predict_class('Puede otra persona retirar mi pedido'))
print(predict_class('hasta luego'))
print(predict_class('adios'))

['información']
['nombre']
['saludo']
['saludo']
['información']
['información']
['despedida']
['despedida']


## 2.3 Si el texto de entrada es del tipo "información”, busque ahora la pregunta mas similar y retorne la respuesta asociada.

In [ ]:
import nltk

nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
pattern = r"""(?x)                   # set flag to allow verbose regexps
              (?:[A-Z]\.)+           # abbreviations, e.g. U.S.A.
              |\$?\d+(?:[.,]\d+)?%?  # numbers, incl. currency and percentages
              |\w+(?:[-']\w+)*       # words w/ optional internal hyphens/apostrophe
              |(?:[+/\-@&*¡!.,])     # special characters with meanings
            """

def normalizer(text, remove_tildes = True): #normalizes a given string to lowercase and changes all vowels to their base form
    text = text.lower() #string lowering
    text = re.sub(r'[^A-Za-zñáéíóú]', ' ', text) #replaces every punctuation with a space
    text = nltk.regexp_tokenize(text, pattern)
    text = ' '.join(text)
    if remove_tildes:
        text = re.sub('á', 'a', text) #replaces special vowels to their base forms
        text = re.sub('é', 'e', text)
        text = re.sub('í', 'i', text)
        text = re.sub('ó', 'o', text)
        text = re.sub('ú', 'u', text)
    return text

In [ ]:
nltk_stopwords = nltk.corpus.stopwords.words('spanish')
stopwords_normalized = [normalizer(word) for word in nltk_stopwords]

def no_stopwords(processed):
  '''
  Función para eliminar las stopwords de un texto
  '''
  result = []
  for w in processed:
    if w not in stopwords_normalized:
      result.append(w)

  return result

In [ ]:
corpus_question = qa.Pregunta.tolist()
questions_processed = []
for question in corpus_question:
  pre = preprocessor(question)
  pre = no_stopwords(pre)
  questions_processed.append(' '.join(pre))

In [ ]:
questions_processed[:5]

['orden transporte ot',
 'plazo retirar envio sucursal chilexpress',
 'puede persona retirar envio sucursal chilexpress',
 'puede modificar direccion envio',
 'puedo cambiar destino envio direccion particular sucursal chilexpress']

ref: https://www.linkedin.com/pulse/how-find-semantic-similar-sentences-from-your-dataset-jayaraman/

In [ ]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize

tagged_data = [TaggedDocument(words=word_tokenize(_d.lower()), tags=[str(i)]) for i, _d in enumerate(questions_processed)]

# hyper parameters
max_epochs = 2000
vec_size = 200
alpha = 0.1
minimum_alpha = 0.025
reduce_alpha = 0.002

model = Doc2Vec(vector_size=vec_size,
                alpha=alpha, 
                min_alpha=minimum_alpha,
                dm =1)
model.build_vocab(tagged_data)

# Train the model based on epochs parameter
for epoch in range(max_epochs):
    model.train(tagged_data,
                total_examples=model.corpus_count,
                epochs=model.iter)

# Save model. 
model.save("similar_sentence.model3")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).


In [ ]:
model= Doc2Vec.load("similar_sentence.model3")

def best_five_answers(question):
  tokens = preprocessor(question)
  tokens = no_stopwords(tokens)
  vector = model.infer_vector(tokens)
  most_similar = model.docvecs.most_similar([vector]) 
  answers = qa.Respuesta.tolist()
  print(tokens)
  for index,value in most_similar[:5]:
    print('---------')
    print(corpus_question[int(index)])
    print(questions_processed[int(index)])
    print(answers[int(index)])
    print('---------')

def best_answers(question):
  tokens = preprocessor(question)
  tokens = no_stopwords(tokens)
  vector = model.infer_vector(tokens)
  most_similar = model.docvecs.most_similar([vector]) 
  answers = qa.Respuesta.tolist()
  for index,value in most_similar[:1]:
    print(corpus_question[int(index)])
    print(answers[int(index)])

In [ ]:
best_five_answers('¿Qué es la Orden de Transporte (OT)?')

['orden', 'transporte', 'ot']
---------
¿Cómo programo los retiros habituales?
programo retiros habituales
Las empresas que poseen una cuenta con Chilexpress pueden solicitar retiros habituales a través su ejecutivo de cuentas asignado.

---------
---------
¿Cuáles son los medios de pago de los servicios online?
cuales medios pago servicios online
Cuando utilizas un servicio online en www.chilexpress.cl, puedes pagar con tarjetas de crédito, débito o transferencia electrónica.

---------
---------
¿Qué es el MTCN?
mtcn
El MTCN es el número de control de envío de dinero, y es único para cada giro. El destinatario necesita conocer el MTCN para retirar el dinero en la sucursal que requiera, mientras que al remitente, le permite hacer seguimiento del envío de dinero.

---------
---------
¿Qué requisito debe cumplir un envío internacional para que sea entregado en el destino?
requisito debe cumplir envio internacional entregado destino
Debes contar con nombre y dirección completa, además de

In [ ]:
best_five_answers('¿Qué es un envío sobredimensionado?')

['envio', 'sobredimensionado']
---------
¿Puedo ir a retirar una encomienda a cualquier oficina de Chilexpress?
puedo ir retirar encomienda cualquier oficina chilexpress
No. Debes retirar tu encomienda o documento en la sucursal donde se encuentra disponible para retiro. Ésta es aquella que fue seleccionada cuando se contrató el servicio.

---------
---------
¿Puedo ir a retirar un documento a cualquier oficina de Chilexpress?
puedo ir retirar documento cualquier oficina chilexpress
No. Debes retirar tu encomienda o documento en la sucursal donde se encuentra disponible para retiro. Ésta es aquella que fue seleccionada cuando se contrató el servicio.

---------
---------
¿Cuál es el plazo para retirar un envío desde una sucursal de Chilexpress?
plazo retirar envio sucursal chilexpress
El plazo para retirar un envío desde una sucursal es de 5 días hábiles. Una vez finalizado dicho plazo, el envío será devuelto al origen. Si el envío se generó en una oficina comercial será devuelto a la 

## 2.4 Reporte el resultado con textos de prueba.

# 3. Análisis transformer 

Para el segundo chatbot utilice un transformer (por ejemplo BERT).

## 3.1 Utilice el modelo transformer para clasificar el texto de entrada, y para extraer la respuesta de la tabla de preguntas y respuestas cuando el mensaje sea del tipo “información”.

## 3.2 Reporte el tipo de red, y las métricas de entrenamiento usadas

## 3.3 Reporte el resultado con los textos de prueba.

# Conclusión